# Lesson 4 – Solutions Notebook  
Conditional Pattern: Customer Support Router Agent

This notebook contains **one possible solution** for the exercises in:

`04_conditional_customer_support_router.ipynb`

Your own solutions may differ but still be valid as long as they implement the required behavior.


## 1. Imports and Extended State Definition

We extend `SupportState` to include:

- `topics: List[str]` – all matched topics (Stretch 1).


In [ ]:
from typing import TypedDict, List

class SupportState(TypedDict):
    user_message: str
    topic: str
    response: str
    topics: List[str]  # all matched topics (may be empty or length 1+)


## 2. Handler Nodes (including Shipping & Language-aware Fallback)

We add a `shipping_handler_node` and a more nuanced `fallback_handler_node` that can mention possible language issues.


In [ ]:
def billing_handler_node(state: SupportState) -> SupportState:
    state["response"] = (
        "Thanks for reaching out about billing. "
        "Please share your invoice number so we can assist."
    )
    return state


def technical_handler_node(state: SupportState) -> SupportState:
    state["response"] = (
        "Thanks for reporting a technical issue. "
        "Please send a screenshot and steps to reproduce."
    )
    return state


def account_handler_node(state: SupportState) -> SupportState:
    state["response"] = (
        "For account issues, please use the password reset link "
        "or confirm your email address so we can help."
    )
    return state


def shipping_handler_node(state: SupportState) -> SupportState:
    state["response"] = (
        "For shipping and delivery questions, please provide your order ID "
        "and current shipping address so we can check the status."
    )
    return state


def fallback_handler_node(state: SupportState) -> SupportState:
    text = state["user_message"].lower()

    # Naive language hint check
    foreign_words = ["bonjour", "hola", "gracias", "merci"]
    if any(w in text for w in foreign_words):
        state["response"] = (
            "We couldn't automatically route your request. "
            "It may be in a language we don't fully support. "
            "A human agent will review your message shortly."
        )
    else:
        state["response"] = (
            "We're not sure which team should handle this. "
            "A human agent will review your message shortly."
        )
    return state


## 3. Classifier Node with Priority and Multi-topic Detection

This classifier implements:

- **Exercise 1:** Adds a `"shipping"` topic using keywords `"shipping"`, `"delivery"`, `"tracking"`.
- **Exercise 2:** Applies explicit priority rules when multiple topics match.
- **Stretch 1:** Populates a `topics: List[str]` field with all matched topics.


In [ ]:
def classify_topic_node(state: SupportState) -> SupportState:
    text = state["user_message"].lower()

    # Determine whether each topic matches
    is_billing = any(w in text for w in ["invoice", "refund", "payment"])
    is_technical = any(w in text for w in ["error", "bug", "crash"])
    is_account = any(w in text for w in ["password", "username", "login"])
    is_shipping = any(w in text for w in ["shipping", "delivery", "tracking"])

    # Collect all matching topics (Stretch 1)
    topics: List[str] = []
    if is_billing:
        topics.append("billing")
    if is_technical:
        topics.append("technical")
    if is_account:
        topics.append("account")
    if is_shipping:
        topics.append("shipping")

    # Naive language hint: if clearly non-English words, route as unknown
    # (Stretch 2)
    foreign_words = ["bonjour", "hola", "gracias", "merci"]
    if any(w in text for w in foreign_words):
        primary_topic = "unknown"
    else:
        # Apply priority among matched topics
        # Example priority: billing > technical > account > shipping
        if is_billing:
            primary_topic = "billing"
        elif is_technical:
            primary_topic = "technical"
        elif is_account:
            primary_topic = "account"
        elif is_shipping:
            primary_topic = "shipping"
        else:
            primary_topic = "unknown"

    state["topic"] = primary_topic
    state["topics"] = topics
    return state


## 4. Wire Up the Conditional LangGraph

We connect:

- Entry: `classify_topic_node`
- Conditional edges:
  - `"billing"` → `billing_handler_node`
  - `"technical"` → `technical_handler_node`
  - `"account"` → `account_handler_node`
  - `"shipping"` → `shipping_handler_node`
  - `"unknown"` → `fallback_handler_node`


In [ ]:
from langgraph.graph import StateGraph, END

graph = StateGraph(SupportState)

graph.add_node("classify_topic", classify_topic_node)
graph.add_node("billing_handler", billing_handler_node)
graph.add_node("technical_handler", technical_handler_node)
graph.add_node("account_handler", account_handler_node)
graph.add_node("shipping_handler", shipping_handler_node)
graph.add_node("fallback_handler", fallback_handler_node)

graph.set_entry_point("classify_topic")


def route_by_topic(state: SupportState):
    return state["topic"]


graph.add_conditional_edges(
    "classify_topic",
    route_by_topic,
    {
        "billing": "billing_handler",
        "technical": "technical_handler",
        "account": "account_handler",
        "shipping": "shipping_handler",
        "unknown": "fallback_handler",
    },
)

# Handlers go to END
graph.add_edge("billing_handler", END)
graph.add_edge("technical_handler", END)
graph.add_edge("account_handler", END)
graph.add_edge("shipping_handler", END)
graph.add_edge("fallback_handler", END)

app = graph.compile()

## 5. Example Runs

Let's test different messages and see how they are routed and handled.


In [ ]:
examples = [
    "I need a refund for my last invoice.",                 # billing
    "My app shows an error when I click save.",           # technical
    "I forgot my password and username.",                 # account
    "Where is my order? I need my delivery tracking.",    # shipping
    "My payment failed and I see an error on checkout.",  # billing + technical
    "Bonjour, je n'arrive pas à me connecter.",           # foreign language hint
]

for msg in examples:
    state: SupportState = {
        "user_message": msg,
        "topic": "",
        "response": "",
        "topics": [],
    }
    result = app.invoke(state)
    print("Message:", msg)
    print("Primary topic:", result["topic"])
    print("All matched topics:", result["topics"])
    print("Response:", result["response"])
    print("-" * 60)


This notebook provides a reference implementation for the **conditional routing pattern** in Lesson 4.

You can adapt the classifier rules, priorities, and responses to match different domains (e.g., internal IT support, HR requests, etc.).
